<a href="https://colab.research.google.com/github/SahajTC/Augmented-Financial-Intelligence/blob/main/LSTMipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import math
import datetime as dt

In [2]:
pip install yfinance prophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 472 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [3]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score,mean_absolute_percentage_error 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM

In [5]:
import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [7]:
import yfinance as yf
btc=yf.download(tickers='BTC-USD', start='2020-01-01', end='2022-11-30', interval = '1d')#bitcoin
eth=yf.download(tickers='ETH-USD', start='2020-01-01', end='2022-11-30', interval = '1d')#etherum
bnb=yf.download(tickers='BNB-USD', start='2020-01-01', end='2022-11-30', interval = '1d')#binance

aapl = yf.download(tickers='AAPL', start='2020-01-01', end='2022-11-30', interval = '1d')#apple
tsla=yf.download(tickers='TSLA', start='2020-01-01', end='2022-11-30', interval = '1d')#tesla
googl=yf.download(tickers='GOOGL', start='2020-01-01', end='2022-11-30', interval = '1d')#google

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [13]:
btc

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,7200.174316,18565664997
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481032
2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,7410.656738,18444271275
2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,7411.317383,19725074095
...,...,...,...,...,...,...
2022-11-25,16602.269531,16603.316406,16388.404297,16521.841797,16521.841797,18678255976
2022-11-26,16521.576172,16666.863281,16416.228516,16464.281250,16464.281250,18000008764
2022-11-27,16463.882812,16594.406250,16437.025391,16444.626953,16444.626953,20443898509


In [12]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]    
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [35]:
def Model_fit(bth):
 bth.reset_index(inplace=True)
 bth['Date'] = pd.to_datetime(bth['Date'],format='%Y-%m-%d')
 closebth = bth[['Date','Close']]
 closebth = closebth[closebth['Date'] > '2020-01-01']

 del closebth['Date']
 scaler=MinMaxScaler(feature_range=(0,1))
 closebth=scaler.fit_transform(np.array(closebth).reshape(-1,1))

 training_size=int(len(closebth)*0.80)
 test_size=len(closebth)-training_size
 train_data,test_data=closebth[0:training_size,:],closebth[training_size:len(closebth),:1]

 time_step = 15
 X_train, y_train = create_dataset(train_data, time_step)
 X_test, y_test = create_dataset(test_data, time_step)
 X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
 X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

 model=Sequential()
 model.add(LSTM(10,input_shape=(None,1),activation="relu"))
 model.add(Dense(1))
 model.compile(loss="mean_squared_error",optimizer="adam")

 history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=32,verbose=1)

 train_predict=model.predict(X_train)
 test_predict=model.predict(X_test)

 train_predict = scaler.inverse_transform(train_predict)
 test_predict = scaler.inverse_transform(test_predict)
 original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
 original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

 print("Train data explained MAPE score:", 
      mean_absolute_percentage_error(original_ytrain, train_predict))
 print("Test data explained MAPE score:", 
      mean_absolute_percentage_error(original_ytest, test_predict))

In [32]:
Model_fit(eth)

Epoch 1/200
27/27 [==============================] - 2s 23ms/step - loss: 0.2420 - val_loss: 0.0620
Epoch 2/200
27/27 [==============================] - 0s 12ms/step - loss: 0.1645 - val_loss: 0.0343
Epoch 3/200
27/27 [==============================] - 0s 10ms/step - loss: 0.1052 - val_loss: 0.0094
Epoch 4/200
27/27 [==============================] - 0s 10ms/step - loss: 0.0363 - val_loss: 0.0084
Epoch 5/200
27/27 [==============================] - 0s 11ms/step - loss: 0.0175 - val_loss: 0.0025
Epoch 6/200
27/27 [==============================] - 0s 11ms/step - loss: 0.0141 - val_loss: 0.0015
Epoch 7/200
27/27 [==============================] - 0s 12ms/step - loss: 0.0121 - val_loss: 0.0012
Epoch 8/200
27/27 [==============================] - 0s 11ms/step - loss: 0.0112 - val_loss: 0.0011
Epoch 9/200
27/27 [==============================] - 0s 12ms/step - loss: 0.0108 - val_loss: 0.0011
Epoch 10/200
27/27 [==============================] - 0s 11ms/step - loss: 0.0101 - val_loss: 0.0012

In [33]:
Model_fit(btc)

Epoch 1/200
27/27 [==============================] - 2s 19ms/step - loss: 0.0254 - val_loss: 0.0034
Epoch 2/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0043 - val_loss: 0.0021
Epoch 3/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0019 - val_loss: 8.1140e-04
Epoch 4/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0013 - val_loss: 5.5232e-04
Epoch 5/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0013 - val_loss: 5.2914e-04
Epoch 6/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0013 - val_loss: 5.5234e-04
Epoch 7/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0013 - val_loss: 5.7717e-04
Epoch 8/200
27/27 [==============================] - 0s 11ms/step - loss: 0.0013 - val_loss: 5.6330e-04
Epoch 9/200
27/27 [==============================] - 0s 11ms/step - loss: 0.0012 - val_loss: 5.2343e-04
Epoch 10/200
27/27 [==============================] - 0s 10ms/step - loss: 0.0

In [34]:
Model_fit(tsla)

KeyError: ignored

In [36]:
Model_fit(aapl)

Epoch 1/200
18/18 [==============================] - 2s 25ms/step - loss: 0.0258 - val_loss: 0.0036
Epoch 2/200
18/18 [==============================] - 0s 10ms/step - loss: 0.0035 - val_loss: 0.0041
Epoch 3/200
18/18 [==============================] - 0s 11ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 4/200
18/18 [==============================] - 0s 11ms/step - loss: 0.0022 - val_loss: 0.0024
Epoch 5/200
18/18 [==============================] - 0s 12ms/step - loss: 0.0019 - val_loss: 0.0023
Epoch 6/200
18/18 [==============================] - 0s 11ms/step - loss: 0.0017 - val_loss: 0.0022
Epoch 7/200
18/18 [==============================] - 0s 11ms/step - loss: 0.0016 - val_loss: 0.0021
Epoch 8/200
18/18 [==============================] - 0s 12ms/step - loss: 0.0014 - val_loss: 0.0020
Epoch 9/200
18/18 [==============================] - 0s 12ms/step - loss: 0.0013 - val_loss: 0.0020
Epoch 10/200
18/18 [==============================] - 0s 12ms/step - loss: 0.0013 - val_loss: 0.0019

In [37]:
Model_fit(googl)

Epoch 1/200
18/18 [==============================] - 2s 23ms/step - loss: 0.7029 - val_loss: 0.4852
Epoch 2/200
18/18 [==============================] - 0s 10ms/step - loss: 0.5261 - val_loss: 0.3657
Epoch 3/200
18/18 [==============================] - 0s 9ms/step - loss: 0.3960 - val_loss: 0.2477
Epoch 4/200
18/18 [==============================] - 0s 9ms/step - loss: 0.2773 - val_loss: 0.1625
Epoch 5/200
18/18 [==============================] - 0s 9ms/step - loss: 0.1722 - val_loss: 0.0729
Epoch 6/200
18/18 [==============================] - 0s 9ms/step - loss: 0.0508 - val_loss: 0.0031
Epoch 7/200
18/18 [==============================] - 0s 9ms/step - loss: 0.0086 - val_loss: 0.0072
Epoch 8/200
18/18 [==============================] - 0s 9ms/step - loss: 0.0063 - val_loss: 0.0053
Epoch 9/200
18/18 [==============================] - 0s 9ms/step - loss: 0.0058 - val_loss: 0.0055
Epoch 10/200
18/18 [==============================] - 0s 10ms/step - loss: 0.0052 - val_loss: 0.0039
Epoch 

In [38]:
Model_fit(bnb)

Epoch 1/200
27/27 [==============================] - 2s 19ms/step - loss: 0.1633 - val_loss: 0.0748
Epoch 2/200
27/27 [==============================] - 0s 10ms/step - loss: 0.0774 - val_loss: 0.0158
Epoch 3/200
27/27 [==============================] - 0s 10ms/step - loss: 0.0238 - val_loss: 0.0012
Epoch 4/200
27/27 [==============================] - 0s 12ms/step - loss: 0.0128 - val_loss: 9.2664e-04
Epoch 5/200
27/27 [==============================] - 0s 11ms/step - loss: 0.0095 - val_loss: 0.0012
Epoch 6/200
27/27 [==============================] - 0s 11ms/step - loss: 0.0077 - val_loss: 0.0020
Epoch 7/200
27/27 [==============================] - 0s 10ms/step - loss: 0.0067 - val_loss: 0.0031
Epoch 8/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0063 - val_loss: 0.0021
Epoch 9/200
27/27 [==============================] - 0s 10ms/step - loss: 0.0058 - val_loss: 0.0030
Epoch 10/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0055 - val_loss: 0.00